# Classifying search terms - in preparation for the 2019 SAB

In this notebook the submitted search terms are classified into a few main categories representing the conceptual units of the GWAS Catalog eg. gene, trait etc. The logfiles are fetched and parsed as described [here](https://www.ebi.ac.uk/seqdb/confluence/pages/viewpage.action?pageId=64720227).

**Analysed period**: 2019.01.01-2019.06.30

**Classes:**

* ancestry
* author
* consortia
* cytoband
* gene
* pmid
* region
* study
* trait
* variant
* unclassified


## Criteria of classes

### Ancestry

If a search term matches a country name or a known ethnic group or ancestry definition. Where do we have the list of such terms:

**1. GWAS Catalog ancestry file from the download [page](https://www.ebi.ac.uk/gwas/docs/file-downloads):**

In [11]:
%%bash

curl -s 'https://www.ebi.ac.uk/gwas/api/search/downloads/ancestry' | cut -f9,10,11 \
    | perl -F"\t" -MData::Dumper -lane '
        foreach $field (@F){
            next if $field eq "NR";
            @elements = split (", ", $field);
            foreach $element (@elements){
                next if $element eq "NR";
                $h{$element} ++;
            }
        }; END {
            print join "\n", keys %h;
        }' | sort -u > ancestry_related_terms_GWAS_Cat.txt
echo "[Info] Number of unique ancestry related terms: " $(cat ancestry_related_terms_GWAS_Cat.txt | wc -l )

[Info] Number of unique ancestry related terms:  157


**2. A full list of countries was also downloaded:**

In [2]:
%%bash 

curl -s https://www.searchify.ca/wp-content/uploads/2016/09/country-keyword-list.csv > country-keyword-list.csv
dos2unix country-keyword-list.csv
echo "[Info] Number of countries: " $(cat country-keyword-list.csv | wc -l)

[Info] Number of countries:  209


dos2unix: converting file country-keyword-list.csv to Unix format ...


**3. A list of all known [ethnicities](https://en.wikipedia.org/wiki/List_of_contemporary_ethnic_groups) from wikipedia**

The table is copy-pasted to a spreadsheet, then extensively cleaned to remove non-standard whitespaces and special characters. As the file contain a huge number of ethnic definition, areas of ethnic prevalences, some misleading terms needs to be excluded: eg. iron. Also there are handful of ethnicities with short names that are happen to be gene names as well.

Parsing the cleaned ancestry file:

In [12]:
%%bash 

tail -n+2 ethnic_groups.txt \
    | perl -F"\t" -lane 'foreach $f (@F){ 
            next if $_ =~ /^\s+$/; 
            foreach $g (split ",", $f){ 
                $g =~ s/^\s*|$\s*//g; 
                $h{$g} = 1 unless length $g < 3; 
            }
        }
        END {
            foreach $g (keys %h){
                print lc $g;
            }
        }' | sort -u | grep -v iron > cleaned_ethnic_groups.txt
        
echo "[Info] Number of unique ethnic groups: " $(cat cleaned_ethnic_groups.txt | wc -l)

[Info] Number of unique ethnic groups:  2317


Merging the previously generated three files:

In [13]:
%%bash

cat ancestry_related_terms_GWAS_Cat.txt \
    country-keyword-list.csv \
    cleaned_ethnic_groups.txt \
    | sort -u > combined_ancestry_related_terms.txt
    
echo "[Info] Combined ancestry related terms:" $(cat combined_ancestry_related_terms.txt | wc -l)

[Info] Combined ancestry related terms: 2574


### Author

Search terms where a word is followed by one or two letters. eg. `Prins B`. The following words are not considered in this classification: vitamin, hepatitis, cystatin, apolipoprotein.

### Consortia

We were interested how many of the queries are directed to consortias, named chorts. Currently a very rough filter is applied: the term is considered to be consortia related if it contains the word `consort` or `biobank`.

### Cytoband

Cytological bands are also a reasonably frequent search terms. Terms that look like this: `xq12.4` or `12p53` are considered as cytological bands.

### Gene

Genes are among the most frequently searched terms. Unfortunately users are search by IDs from different databases, synonyms etc. Dictionary with all known gene names/IDs are generated as follows:

In [8]:
%%bash 

# Downloading GENCODE data:
wget -q ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_31/gencode.v31.annotation.gtf.gz

# Downloading Entrez gene info:
wget -q ftp://ftp.ncbi.nih.gov/gene/DATA/GENE_INFO/Mammalia/Homo_sapiens.gene_info.gz

cat <(zless gencode.v31.annotation.gtf.gz | grep -v "#"| awk '$3 == "gene"' \
   | perl -lane 'print $1 if $_ =~ /gene_name "(.+?)"/' )\
   <(zless Homo_sapiens.gene_info.gz  | grep -v "#" | perl -lane 'print  join "\n", $F[2], split(/\|/, $F[4])') \
   | perl -lane '$_ =~ s/\.\d+$//; $_ =~ s/\.$//; print $_ if length($_) >= 3' | sort -u > gene_list.txt

echo "[Info] Combined gene term list:" $(cat gene_list.txt | wc -l)

[Info] Combined gene term list: 134098


A search term is considered to be gene if matches to a gene term or Ensembl gene ID (`ENSG00...`) or gene card ID (`gc....`). 

### Pubmed ID

Finding pubmed ID is not easy. There are a lot of numbers in general: genomic region, rsID, gene ID etc. So a number is considered to be a pubmed ID if it has only 8 digits starting with 1,2 or 3. Does not have `rs`, `kgp`, `exm`, `ensg` or does not look like a chromosome: `X:2039203`.

### Genomic region

A search term is considered to be a genomic region if it has a chromosome (`1..22`,`x`,`y`) followed by `:`, number, then `-` then number again.

### GWAS Catalog study

A search term is considered to be study if it looks like this: `GCST####`. 

### Trait

Identifying traits is tricky. First we generate a large dictionary of possible trait names and look up the search terms.

In [9]:
%%bash

# Extracting all traits, reported traits, synonyms etc:
curl -s "http://garfield.ebi.ac.uk:8983/solr/gwas_slim/select?q=resourcename%3Atrait&rows=2467&fl=parent%2CmappedTrait%2C+synonyms%2C+reportedTrait&wt=json&indent=true" | jq -r '.response.docs' > traits.json


# Reading json in perl and get a unique list of all the trait terms:
./trait_parser.pl traits.json | sort > unique_trait_terms.lst

# Generating a list of words typical for phenotypes:
cat unique_trait_terms.lst <(echo "als aids area acne age stroke measurement bmi barr blue body levels bone copd cold deep disc down ear eye egg fast fear food fat \
    foot free grout gray hiv hpv hair hand head height high hip lead lean iris iron ratio itch knee left lewy life lip \
    lobe low lung levels mmr mri male mean activity status adenocarcinoma cinoma mild milk mood oral pet pain peak poor post rate red sex skin stem sum type \
    urea uric vein very zinc ward word acid air area aura band base beta dose drop diet born cell drug dust eyes \
    face flow gain germ grip hay hand hair heel heme jaw late legs load meal anger agression parkinsons \
    body bmi adhd measure rheumatoid alzheimer renal crohn fat factor gamma height Parkinsons hyperinsulinemia \
    hiv hepatitis hypertension pulmonary hdl serum schizophrenia sensitivity hyperlipidemia dyslipidemia IQ \
    ldl skin stroke  stomach systemic substance systolic survival crohns ptsd lactase salt retinoblastom \
    level total thyroid thioredoxin concentrations viral als event waist vitamin vitemin birthweight \
    vitami vitamine vitamina density function Response disease symptom degeneration hemophilia \
    chron failure cancer obesity ibd sarcoma blood lymphoma association risk syndrome deafness breastcancer \
    disorder type diabetes t2d t1d" | tr " " "\n" ) | sort -u | awk 'NF != 0' | sponge unique_trait_terms.lst
    
echo "[Info] Size of the combined term list:" $(cat unique_trait_terms.lst | wc -l)

[Info] Size of the combined term list: 14826


Wide character in print at ./trait_parser.pl line 42.
Wide character in print at ./trait_parser.pl line 42.
Wide character in print at ./trait_parser.pl line 42.
Wide character in print at ./trait_parser.pl line 42.
Wide character in print at ./trait_parser.pl line 42.
Wide character in print at ./trait_parser.pl line 42.
Wide character in print at ./trait_parser.pl line 42.
Wide character in print at ./trait_parser.pl line 42.
Wide character in print at ./trait_parser.pl line 42.


### variant

Variants are marked by several different ways: rsID, KGP, EXM ID or chromosome:position.


### Unclassified

A term is unclassified if none of the above critera was satisfied.

## Exclusion

Queries we are displaying on the front search page were omitted from the counting:

* `breast carcinoma`
* `rs7329174`
* `Yao`
* `2q37.1`
* `HBS1L` 
* `6:16000000-25000000`

## Calling classifier


```bash
./search_classifier.pl \
    -geneFile gene_list.txt \
    -traitFile unique_trait_terms.lst  \
    -ancestryFile combined_ancestry_related_terms.txt \
    -logFile search_2019-01-01-2019-06-31_queries.tsv \
    -outputFolder output
```

## Input set

To generate the input set, the logfiles are parsed and pooled to get a unique list of search terms and the number how many times are term was queried:

```bash
head search_2019-01-01-2019-06-31_queries.tsv
```

```
177597  rs7329174
5238    breast%20carcinoma
1468    body%20mass%20index
1157    schizophrenia
1091    diabetes
1018    multiple%20sclerosis
1009    obesity
944     asthma
937     breast%20cancer
928     Alzheimer%27s%20disease
````

* The number of queries in the requested period: 974,599
* Number of unique search terms: 209,853

## Output

The search terms are classified into the above described bins. Some terms are assigned to multiple bins. The table below shows the overlap:

| Number of bins | Count | Percent |
|:------|:------|:------|
| 1 | 958422 | 98.34014% |
| 2 | 16169 | 1.65904% |
| 3 | 7 | 0.00072% |
| 4 | 1 | 0.00010% |

As the table shows 98.3% of all queries (not terms!) were binned into one single class. That's very reassuring. 

The distrubution of queries across categories look like this:

| Bin | Count | Percent |
|:------|:------|:------|
| variant | 344309 | 35.328% |
| gene | 383378 | 39.337% |
| trait | 156029 | 16.010% |
| unclassified | 57953 | 5.946% |
| pmid | 30009 | 3.079% |
| author | 10050 | 1.031% |
| cytoband | 3002 | 0.308% |
| region | 2948 | 0.302% |
| study | 2590 | 0.266% |
| ancestry | 287 | 0.029% |
| consortia | 230 | 0.024% |

![pie_chart]( pie_chart.png "Distribution ofsearches")